In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
## load the model
model = load_model("model.h5")

## load the scaler and encoders
with open("scaler.pkl","rb") as f:
    scaler = pickle.load(f)

with open("label_encoder_gender.pkl","rb") as f:
    label_encoder_gender = pickle.load(f)

with open("onehot_encoder_geo.pkl","rb") as f:
    onehot_encoder_geo = pickle.load(f)

In [3]:
## Example data point
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [11]:
## one hot encode the geography column
geo_encoded = np.array(onehot_encoder_geo.transform([[input_data['Geography']]]))
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))


c:\Users\pradh\Pradhan\KrishNaik\ANN\annpractice\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [12]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
## create dataframe from input data point
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df,geo_encoded_df],axis=1)
input_df.drop(columns=['Geography'],inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
## Scale the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [ ]:
## Predict the class
pred = model.predict(input_scaled)


1/1 [==============================] - 0s 194ms/step
Predicted class: 0, Probability of Churn: 6.22%


In [18]:
pred_prob = pred[0][0]
if pred_prob > 0.5:
    print(f"Customer will leave the bank with a probability of {pred_prob}")
else:
    print(f"Customer will stay with the bank with a probability of {(1 - pred_prob):.2f}")

Customer will stay with the bank with a probability of 0.94


In [ ]:
pred_class = (pred > 0.5).astype("int32")
print(f"Predicted class: {pred_class[0][0]}, Probability of Churn: {pred[0][0]*100:.2f}%")